# Pair-aware LIME for Pairwise Text Classification

This notebook demonstrates the differences between standard LIME and our pair-aware LIME implementation for explaining pairwise text classification models (e.g., authorship verification).

## The Problem

Standard LIME perturbations can only control **dissimilarity** between texts:
- Removing a word with different frequency in two texts → texts become more similar
- Removing a word with similar frequency → similarity unchanged (both texts shift together)

This means standard LIME can explain class 0 (different authors) but struggles with class 1 (same author).

## The Solution

Pair-aware perturbations can control **similarity** by perturbing each text independently:
- Removing a word only from one text → breaks any similarity that existed

This enables meaningful explanations for both classes.

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '.')

import numpy as np

# Import our custom module
from lime_pair import PairIndexedString

# Try to import the full explainer (requires lime package)
try:
    from lime_pair import PairLimeTextExplainer
    from lime.lime_text import LimeTextExplainer
    LIME_AVAILABLE = True
    print("LIME package available - full functionality enabled")
except ImportError:
    LIME_AVAILABLE = False
    print("LIME package not available - showing core concepts only")

## 2. Understanding PairIndexedString

The key innovation is building **separate vocabularies** for each text segment.

In [ ]:
# Example text pair with separator
text_pair = '''Rinoa let out a soft giggle. "Okay Uncle Laguna." $&*&*&$"As always, make yourselves at home!"'''

print("Original text pair:")
print(text_pair)
print()
print("Left segment:", text_pair.split('$&*&*&$')[0])
print("Right segment:", text_pair.split('$&*&*&$')[1])

In [ ]:
# Create indexed string with dual vocabularies
idx_str = PairIndexedString(text_pair, bow=True)

print("PairIndexedString representation:")
print(idx_str)
print()
print(f"Left vocabulary size: {idx_str.num_words_left()}")
print(f"Right vocabulary size: {idx_str.num_words_right()}")
print(f"Total features: {idx_str.num_words()}")
print(f"Separator index: {idx_str.get_separator_index()}")

In [ ]:
# Examine the dual vocabularies
print("LEFT VOCABULARY:")
print(f"  vocab_left: {idx_str.vocab_left}")
print(f"  inverse_vocab_left: {idx_str.inverse_vocab_left}")
print()
print("RIGHT VOCABULARY:")
print(f"  vocab_right: {idx_str.vocab_right}")
print(f"  inverse_vocab_right: {idx_str.inverse_vocab_right}")

### Key Observation

Notice that even though words may appear in both texts, they are **separate features** in our dual vocabulary system. This allows independent perturbation.

## 3. Segment-Aware Word Removal

The `inverse_removing` method can remove words from only one segment.

In [ ]:
# Original text
print("ORIGINAL:")
print(idx_str.raw_string())
print()

# Remove first word from LEFT segment only
left_indices_to_remove = [0]  # 'Rinoa' in left vocab
result_left = idx_str.inverse_removing(left_indices_to_remove, segment='left')
print(f"AFTER REMOVING '{idx_str.inverse_vocab_left[0]}' FROM LEFT:")
print(result_left)
print()

# Remove first word from RIGHT segment only  
right_indices_to_remove = [0]  # First word in right vocab
result_right = idx_str.inverse_removing(right_indices_to_remove, segment='right')
print(f"AFTER REMOVING '{idx_str.inverse_vocab_right[0]}' FROM RIGHT:")
print(result_right)

In [ ]:
# Demonstrate that removing from one segment doesn't affect the other
print("Demonstrating segment isolation:")
print()

# Remove multiple words from left
result = idx_str.inverse_removing([0, 1, 2], segment='left')
print("Remove indices [0,1,2] from LEFT:")
print(f"  Removed: {[idx_str.inverse_vocab_left[i] for i in [0,1,2]]}")
print(f"  Result: {result}")
print()

# Verify right segment is unchanged
right_part = result.split('$&*&*&$')[1]
original_right = text_pair.split('$&*&*&$')[1]
print(f"  Right segment unchanged: {right_part == original_right}")

## 4. Comparison with Standard LIME Approach

In standard LIME, the same word appearing in both texts would be a **single feature**. Removing it would affect both texts simultaneously, making it impossible to control similarity.

In [ ]:
# Simulated example of the difference
text_with_shared_word = 'The cat sat $&*&*&$ The cat ran'

idx = PairIndexedString(text_with_shared_word, bow=True)

print("Text:", text_with_shared_word)
print()
print("In PAIR-AWARE LIME:")
print(f"  Left vocab: {idx.vocab_left}")
print(f"  Right vocab: {idx.vocab_right}")
print()
print("Notice 'The' and 'cat' appear in BOTH vocabularies as separate features.")
print()

# In standard LIME, 'cat' would be ONE feature
# Removing it would remove from BOTH texts
print("STANDARD LIME would have:")
print("  Single vocab: {'The': 0, 'cat': 1, 'sat': 2, 'ran': 3}")
print("  Removing 'cat' removes from BOTH texts simultaneously")
print()

# Our approach
print("PAIR-AWARE LIME:")
print(f"  Remove 'cat' from LEFT only: {idx.inverse_removing([idx.vocab_left['cat']], segment='left')}")
print(f"  Remove 'cat' from RIGHT only: {idx.inverse_removing([idx.vocab_right['cat']], segment='right')}")

## 5. Perturbation Modes

The `PairLimeTextExplainer` supports three modes:

| Mode | Description | Best For |
|------|-------------|----------|
| `'left'` | Only perturb left text | Analyzing left text's contribution to similarity |
| `'right'` | Only perturb right text | Analyzing right text's contribution to similarity |
| `'rand'` | Randomly choose side per sample | General exploration (non-BOW only) |

In [ ]:
# Demonstrate perturbation behavior conceptually
text = 'Hello world $&*&*&$ Goodbye world'
idx = PairIndexedString(text, bow=True)

print("Original:", text)
print()
print("LEFT mode perturbations (right segment always intact):")
for i in range(3):
    # Randomly remove some words from left
    np.random.seed(i)
    n_remove = np.random.randint(1, idx.num_words_left())
    to_remove = np.random.choice(idx.num_words_left(), n_remove, replace=False)
    result = idx.inverse_removing(to_remove.tolist(), segment='left')
    print(f"  Sample {i+1}: {result}")

print()
print("RIGHT mode perturbations (left segment always intact):")
for i in range(3):
    np.random.seed(i)
    n_remove = np.random.randint(1, idx.num_words_right())
    to_remove = np.random.choice(idx.num_words_right(), n_remove, replace=False)
    result = idx.inverse_removing(to_remove.tolist(), segment='right')
    print(f"  Sample {i+1}: {result}")

## 6. Using PairLimeTextExplainer (requires lime package)

In [ ]:
if LIME_AVAILABLE:
    # Mock classifier for demonstration
    def mock_classifier(texts):
        """Simple mock classifier based on text length difference."""
        probs = []
        for text in texts:
            parts = text.split('$&*&*&$')
            if len(parts) == 2:
                # Similar lengths → higher class 1 probability
                len_diff = abs(len(parts[0]) - len(parts[1]))
                p1 = max(0.1, min(0.9, 1 - len_diff / 100))
            else:
                p1 = 0.5
            probs.append([1 - p1, p1])
        return np.array(probs)
    
    # Create explainers for both modes
    explainer_left = PairLimeTextExplainer(mode='left', bow=True, random_state=42)
    explainer_right = PairLimeTextExplainer(mode='right', bow=True, random_state=42)
    
    text_pair = 'The quick brown fox jumps over $&*&*&$ A quick brown dog runs under'
    
    print("Text pair:", text_pair)
    print()
    
    # Get explanations
    exp_left = explainer_left.explain_instance(
        text_pair, mock_classifier, 
        num_samples=500, num_features=5, labels=(0, 1)
    )
    exp_right = explainer_right.explain_instance(
        text_pair, mock_classifier,
        num_samples=500, num_features=5, labels=(0, 1)
    )
    
    print("LEFT mode explanation (class 1 - same author):")
    print(f"  Intercept: {exp_left.intercept[1]:.4f}")
    print(f"  Top features: {exp_left.as_list(label=1)[:5]}")
    print()
    print("RIGHT mode explanation (class 1 - same author):")
    print(f"  Intercept: {exp_right.intercept[1]:.4f}")
    print(f"  Top features: {exp_right.as_list(label=1)[:5]}")
else:
    print("LIME package not available. Install with: pip install lime")
    print("Then re-run this cell to see the full explainer demonstration.")

## 7. Recommended Usage Pattern

Based on our research findings:

```python
from lime.lime_text import LimeTextExplainer
from lime_pair import PairLimeTextExplainer

# For class 0 (different authors) - use standard LIME
standard_explainer = LimeTextExplainer(bow=True)

# For class 1 (same author) - use pair-aware LIME with both modes
explainer_left = PairLimeTextExplainer(mode='left', bow=True)
explainer_right = PairLimeTextExplainer(mode='right', bow=True)

# Generate explanations
if predicted_class == 0:
    exp = standard_explainer.explain_instance(text_pair, classifier_fn)
else:  # class 1
    exp_left = explainer_left.explain_instance(text_pair, classifier_fn)
    exp_right = explainer_right.explain_instance(text_pair, classifier_fn)
    # Combine features from both explanations
```

## 8. Summary

### Standard LIME Limitations for Pairwise Classification
- Uses single vocabulary across entire input
- Perturbations affect both texts simultaneously
- Can only control dissimilarity (explain class 0)
- High intercept, low feature weights for class 1

### Pair-aware LIME Advantages
- Separate vocabularies for each text segment
- Perturbations can target individual segments
- Can control similarity (explain class 1)
- Meaningful feature weights for both classes

### When to Use Each
| Predicted Class | Explanation Goal | Use |
|-----------------|------------------|-----|
| 0 (different) | Find dissimilarity features | Standard LIME |
| 1 (same) | Find similarity features | Pair-aware LIME (left + right) |